In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from MCMAC_pre_merger import MCengine_pre_merger
from MCMAC_post_merger import MCengine_post_merger 

import numpy as np
import pickle

# Input A2034 parameters

In [2]:
# Setting simulation parameters (define prior distributions)
N_mc = 1000  # Number of Monte Carlo samples

cluster_name ='A2034'

arcsecond = 300
redshift = 0.114

# (mean value, standard deviation)
M1 = (2.35e14, 0.84e14)  # Mass distribution of A2034NW (Subcluster 1)
M2 = (1.08e14, 0.51e14)  # Mass distribution of A2034NE (Subcluster 2)
Z1 = (0.114, 0)  # Redshift distribution of A2034NW (Subcluster 1)
Z2 = (0.114, 0)  # Redshift distribution of A2034NE (Subcluster 2)

TSC_simulation = 0.26
TSC_simulation_lower = 0
TSC_simulation_upper = 0

# when you use pre-MCMAC code
# prefix = "../RealClusters/A2034/simulation_pre_output" 

# when you use post-MCMAC code
prefix = "../RealClusters/A2034/simulation_post_output" 

result_file = "result_A2034.csv"

## arcsecond to kpc

In [3]:
from astropy.cosmology import Planck18 as cosmo

def arcsecondToKpc(redshift, arcsecond):
    # Convert arcseconds to kiloparsecs
    kpc_per_arcsec = cosmo.kpc_proper_per_arcmin(redshift).value / 60  # kpc per arcsec
    distance_kpc = arcsecond * kpc_per_arcsec
    return distance_kpc

In [4]:

distance_Mpc = arcsecondToKpc(redshift, arcsecond) * 0.001 # kpc to Mpc
print(distance_Mpc)

D_proj = (distance_Mpc, 0.05*distance_Mpc, 0.05*distance_Mpc)  # Projected separation distance distribution (mean 408 kpc, standard deviation 14 kpc)

0.6415971364028084


## Confirm it collision or not

In [5]:
# Execute simulation (derive posterior distributions of dynamical properties such as collision velocity, time since collision, etc.)
# MCengine_pre_merger(N_mc, M1, M2, Z1, Z2, D_proj, prefix)

Probability of system being bound = 1.00

Probability of system being boud is over 50%, then use post merger code.

In [6]:
MCengine_post_merger(N_mc, M1, M2, Z1, Z2, D_proj, prefix)

Completed Monte Carlo iteration 10 of 1000.
~3 minutes remaining
Completed Monte Carlo iteration 100 of 1000.
~3 minutes remaining
Completed Monte Carlo iteration 1000 of 1000.
~0 minutes remaining


(array([2.94615445e+14, 2.66333274e+14, 2.62608681e+14, 2.83808312e+14,
        2.65828685e+14, 2.46975876e+14, 1.76233654e+14, 2.19286168e+14,
        2.30788508e+14, 1.16400614e+14, 2.17297463e+14, 1.89680592e+14,
        1.75827695e+14, 2.62986183e+14, 3.41380681e+14, 1.66372996e+14,
        2.50086222e+14, 3.08364751e+14, 9.94017258e+13, 1.47474782e+14,
        2.32040456e+14, 2.75538561e+14, 2.83625357e+14, 3.14834930e+14,
        4.07984907e+14, 1.95303822e+14, 2.90545118e+14, 2.83149178e+14,
        2.08532998e+14, 2.69813422e+14, 1.15929606e+14, 2.40320130e+14,
        4.15037954e+13, 1.37180218e+14, 5.98804811e+13, 2.01170647e+14,
        2.38681663e+14, 1.25407630e+14, 1.84756180e+14, 1.96250563e+14,
        5.97846597e+13, 2.94109398e+14, 2.61493753e+14, 2.05252028e+14,
        2.70436034e+14, 2.71795710e+14, 2.08431962e+14, 2.90850744e+14,
        1.14366568e+14, 1.88094297e+14, 1.85857213e+14, 2.26206467e+14,
        1.91175521e+14, 2.67106844e+14, 3.39803454e+14, 2.722290

In [7]:
import pickle
import numpy as np
import pandas as pd

# 単位の定義
units = {
    'm1': '10^14 M☉',
    'm2': '10^14 M☉',
    'z1': '',
    'z2': '',
    'dproj': 'Mpc',
    'vradobs': 'km s^{-1}',
    'alpha': 'degree',
    'v3dobs': 'km s^{-1}',
    'd3d': 'Mpc',
    'v3dcol': 'km s^{-1}',
    'dmax': 'Mpc',
    'TSM0': 'Gyr',
    'TSM1': 'Gyr',
    'T': 'Gyr',
    'prob': ''
}

# パラメータファイルのリスト
parameters = ['_m_1.pickle', '_m_2.pickle', '_z_1.pickle', '_z_2.pickle', '_d_proj.pickle', 
              '_v_rad_obs.pickle', '_alpha.pickle', '_v_3d_obs.pickle', '_d_3d.pickle', 
              '_v_3d_col.pickle', '_d_max.pickle', '_TSM_0.pickle', '_TSM_1.pickle', 
              '_T.pickle', '_prob.pickle']

results = []

for param in parameters:
    with open(prefix + param, 'rb') as f:
        data = pickle.load(f)
        if param in ['_m_1.pickle', '_m_2.pickle']:
            # 質量をe14単位に変換
            data = np.array(data) / 1e14
        
        # Medianを計算
        median = np.median(data)
        
        # 68%および95%の信頼区間を計算
        lcl_68, ucl_68 = np.percentile(data, [16, 84])
        lcl_95, ucl_95 = np.percentile(data, [2.5, 97.5])
        
        param_name = param.replace('_', '').replace('.pickle', '')
        
        # 数値を有効数字2桁にフォーマット
        if param_name not in ['z_1', 'z_2']:
            median = round(median, 2)
            lcl_68 = round(lcl_68, 2)
            ucl_68 = round(ucl_68, 2)
            lcl_95 = round(lcl_95, 2)
            ucl_95 = round(ucl_95, 2)
        
        results.append({
            'Parameter': param_name,
            'Units': units[param_name],
            'Median': median,
            '68% LCL': lcl_68,
            '68% UCL': ucl_68,
            # '95% LCL': lcl_95,
            # '95% UCL': ucl_95
        })

# 結果をDataFrameに変換
df_results = pd.DataFrame(results)
display(df_results)

# TSM0とTSM1に対応する行の抽出
tsm0_row = df_results[df_results['Parameter'] == 'TSM0']
tsm1_row = df_results[df_results['Parameter'] == 'TSM1']

# CSVに出力するための新しい行を作成
new_row = {
    'ClusterName': cluster_name,
    'MCMAC_TSC0': tsm0_row['Median'].values[0],
    'MCMAC_TSC0.lower': tsm0_row['68% LCL'].values[0],
    'MCMAC_TSC0.upper': tsm0_row['68% UCL'].values[0],
    'MCMAC_TSC1': tsm1_row['Median'].values[0],
    'MCMAC_TSC1.lower': tsm1_row['68% LCL'].values[0],
    'MCMAC_TSC1.upper': tsm1_row['68% UCL'].values[0],
    'Simulation_TSC': 0.27, 
    'Simulation_TSC.lower': 0,
    'Simulation_TSC.upper': 0
}

# 新しい行をDataFrameに変換
new_row_df = pd.DataFrame([new_row])

# CSVファイルに出力
new_row_df.to_csv(result_file, index=False)


,Parameter,Units,Median,68% LCL,68% UCL
0,m1,10^14 M☉,2.35,1.52,3.17
1,m2,10^14 M☉,1.07,0.59,1.58
2,z1,,0.11,0.11,0.11
3,z2,,0.11,0.11,0.11
4,dproj,Mpc,0.64,0.61,0.67
5,vradobs,km s^{-1},0.00,0.00,0.00
6,alpha,degree,21.11,2.56,60.23
7,v3dobs,km s^{-1},0.00,0.00,0.00
8,d3d,Mpc,0.70,0.64,1.28
9,v3dcol,km s^{-1},1310.42,1202.48,1701.80
